<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-03 14:03:42
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: 2.87 L (2.1%)
Current PnL: -30.22 L (-19.27%)
CY Booked + Current PnL: -15.48 L (-9.87%)
-------------------
Total profit:  1.46 L
Total loss:  -31.68 L
-------------------
Total Booked + Current PnL: 11.50 L (8.86%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.58%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.73 L (68.74%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.39,4.55,14.04,19.23,22086.0,6846.0,157306.0,484.83,22.81,52.0,L-LC,12.49,182.0,0.31,1.13,26.79,XY25,ATH,MINING
50,MASFIN,1.73,-3.80,26.87,22.05,25328.0,-3720.0,94260.0,398.61,-16.89,50.0,H-SC,3.47,168.0,-0.15,0.68,37.01,XR,ATH,FINANCE
77,TTKPRESTIG,0.44,-27.14,37.38,0.09,27447.0,-27351.0,73426.0,770.00,70.33,35.0,M-SC,7.12,253.0,-1.00,0.53,0.44,OX40N,NTT,DURABLES
33,HCLTECH,1.44,10.39,12.18,23.83,32528.0,25133.0,267057.0,1908.19,15.30,57.0,X-LC,7.65,13.0,0.77,1.92,28.13,X40,ATH,IT
51,MEDANTA,-2.40,-7.08,33.54,24.08,39563.0,-8993.0,117957.0,1486.00,-14.63,48.0,X-SC,10.32,91.0,-0.23,0.85,8.04,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,11.77,-22.95,108.66,60.77,153020.0,-41950.0,140825.0,521.0,62.53,65.0,H-SC,4.45,158.0,-0.27,1.01,27.67,XR,NTT,JEWELLERY
76,TRIDENT,11.71,-22.86,66.61,28.51,47400.0,-21094.0,71161.0,48.0,-8.72,67.0,M-SC,10.72,226.0,-0.45,0.51,21.77,XR,NTT,TEXTILES
46,JIOFIN,8.15,-15.08,46.59,24.48,105778.0,-40325.0,227040.0,387.0,-16.52,42.0,X-LC,8.53,37.0,-0.38,1.63,31.60,XY24,BTT,FINANCE
5,ANGELONE,7.66,5.86,17.32,24.20,55970.0,17900.0,323150.0,3033.0,50.86,54.0,X-SC,2.00,99.0,0.32,2.32,35.34,X40N,NTT,FINANCE
56,RAJOOENG,6.79,-37.81,124.51,39.61,79363.0,-38760.0,63740.0,143.1,-57.95,46.0,H-SC,28.26,136.0,-0.49,0.46,6.79,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JCHAC,-3.77,-42.19,72.96,-0.01,53907.0,-53919.0,73886.0,2282.0,15030.43,39.0,M-SC,15.50,235.0,-1.00,0.53,2.44,OX40N,BTT,AC
48,KANSAINER,-2.45,-23.76,48.84,13.47,100409.0,-64080.0,205587.0,340.0,-68.70,47.0,H-SC,5.12,159.0,-0.64,1.48,4.54,XY24,NTT,PAINTS
51,MEDANTA,-2.40,-7.08,33.54,24.08,39563.0,-8993.0,117957.0,1486.0,-14.63,48.0,X-SC,10.32,91.0,-0.23,0.85,8.04,XY24,NTT,HEALTHCARE
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.0,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
2,ABBOTINDIA,-1.35,-9.57,30.57,18.07,49441.0,-17125.0,161730.0,35195.0,-22.48,39.0,X-MC,8.96,64.0,-0.35,1.16,5.21,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,2.38,0.64,19.97,20.74,40831.0,1292.0,204462.0,446.0,33.49,54.0,M-MC,4.54,189.0,0.03,1.47,11.32,XY25,NTT,FINANCE
84,WIPRO,0.56,-0.07,72.38,72.27,122566.0,-111.0,169337.0,420.0,-14.25,48.0,M-LC,2.62,101.0,-0.00,1.22,8.85,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.51,-6.15,121.53,107.91,168400.0,-9077.0,138567.0,1641.55,-23.79,49.0,M-SC,11.27,220.0,-0.05,1.00,4.25,OX40N,ATH,CABLES
72,TATAELXSI,0.34,-21.33,67.83,32.03,70348.0,-28123.0,103712.0,9161.00,-14.86,52.0,H-SC,6.50,157.0,-0.40,0.74,16.17,OX40N,NTT,IT
66,SIS,0.47,-21.90,55.27,21.26,47926.0,-24319.0,86713.0,528.00,2048.70,59.0,H-SC,4.96,163.0,-0.51,0.62,17.16,OX40N,NTT,MISC
48,KANSAINER,-2.45,-23.76,48.84,13.47,100409.0,-64080.0,205587.0,340.00,-68.70,47.0,H-SC,5.12,159.0,-0.64,1.48,4.54,XY24,NTT,PAINTS
40,INDIGOPNTS,-0.16,-26.78,36.59,0.02,46769.0,-46740.0,127819.0,1408.00,94.75,30.0,M-SC,5.23,240.0,-1.00,0.92,11.18,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,2.38,0.64,19.97,20.74,40831.0,1292.0,204462.0,446.00,33.49,54.0,M-MC,4.54,189.0,0.03,1.47,11.32,XY25,NTT,FINANCE
18,COALINDIA,0.39,4.55,14.04,19.23,22086.0,6846.0,157306.0,484.83,22.81,52.0,L-LC,12.49,182.0,0.31,1.13,26.79,XY25,ATH,MINING
1,ABB,3.81,4.15,39.77,45.57,108362.0,10853.0,272472.0,7934.00,-33.07,70.0,H-MC,6.08,121.0,0.10,1.96,21.00,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,3.81,4.15,39.77,45.57,108362.0,10853.0,272472.0,7934.00,-33.07,70.0,H-MC,6.08,121.0,0.10,1.96,21.00,AR,NTT,ELECTRICAL
84,WIPRO,0.56,-0.07,72.38,72.27,122566.0,-111.0,169337.0,420.00,-14.25,48.0,M-LC,2.62,101.0,-0.00,1.22,8.85,XR,NTT,IT
39,INDIAMART,2.82,-2.35,111.70,106.72,134528.0,-2899.0,120437.0,4810.62,-51.58,57.0,H-SC,8.90,138.0,-0.02,0.86,20.48,AR,ATH,MISC
26,FINCABLES,2.51,-6.15,121.53,107.91,168400.0,-9077.0,138567.0,1641.55,-23.79,49.0,M-SC,11.27,220.0,-0.05,1.00,4.25,OX40N,ATH,CABLES
85,ZYDUSLIFE,2.51,-4.91,42.63,35.63,84956.0,-10287.0,199287.0,1286.17,-18.51,52.0,H-MC,5.82,119.0,-0.12,1.43,10.95,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.00,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
6,ASIANPAINT,0.99,-15.21,42.64,20.95,91027.0,-38289.0,213479.0,3460.25,-16.09,27.0,X-LC,10.22,36.0,-0.42,1.53,15.01,X40,ATH,PAINTS
12,BATAINDIA,0.52,-44.01,141.47,35.20,101921.0,-56626.0,72044.0,2096.00,-5.56,31.0,X-SC,20.42,93.0,-0.56,0.52,2.38,X40,NTT,FOOTWEAR
13,BERGEPAINT,2.54,-14.85,42.26,21.14,81811.0,-33749.0,193590.0,680.00,-20.83,33.0,X-MC,7.73,75.0,-0.41,1.39,3.26,XY24,NTT,PAINTS
53,PGHH,0.65,-11.10,50.94,34.19,96696.0,-23696.0,189824.0,17907.65,-37.47,33.0,X-MC,8.04,60.0,-0.25,1.36,0.94,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,1.54,-11.51,33.97,18.55,113699.0,-43534.0,334703.0,4311.58,-23.62,56.0,X-LC,0.63,3.0,-0.38,2.40,13.87,X40,ATH,IT
81,VBL,-1.05,-5.81,44.04,35.67,131019.0,-18343.0,297499.0,671.64,-17.02,39.0,X-LC,1.51,5.0,-0.14,2.14,7.24,X40N,ATH,FMCG
5,ANGELONE,7.66,5.86,17.32,24.20,55970.0,17900.0,323150.0,3033.00,50.86,54.0,X-SC,2.00,99.0,0.32,2.32,35.34,X40N,NTT,FINANCE
34,HINDUNILVR,1.17,-7.23,22.52,13.67,53708.0,-18573.0,238490.0,2922.00,-14.11,53.0,X-LC,2.26,9.0,-0.35,1.71,12.59,XY25,NTT,FMCG
21,DABUR,0.57,-1.88,46.46,43.70,113082.0,-4671.0,243397.0,735.00,-9.97,43.0,X-MC,2.76,73.0,-0.04,1.75,12.88,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.00,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
53,PGHH,0.65,-11.10,50.94,34.19,96696.0,-23696.0,189824.0,17907.65,-37.47,33.0,X-MC,8.04,60.0,-0.25,1.36,0.94,X40,ATH,FMCG
12,BATAINDIA,0.52,-44.01,141.47,35.20,101921.0,-56626.0,72044.0,2096.00,-5.56,31.0,X-SC,20.42,93.0,-0.56,0.52,2.38,X40,NTT,FOOTWEAR
3,ACC,2.57,-29.48,132.78,64.15,222805.0,-70151.0,167800.0,3906.00,-58.93,43.0,X-SC,7.57,84.0,-0.31,1.21,2.57,XY24,BTT,CEMENT
52,PAGEIND,2.37,-15.11,53.86,30.62,108388.0,-35814.0,201240.0,51605.08,-39.68,41.0,X-MC,20.07,57.0,-0.33,1.45,3.14,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.44,-28.57,99.98,42.84,46423.0,-18574.0,46432.0,424.00,-53.17,56.0,X-SC,21.66,83.0,-0.40,0.33,9.14,XY24,NTT,MISC
12,BATAINDIA,0.52,-44.01,141.47,35.20,101921.0,-56626.0,72044.0,2096.00,-5.56,31.0,X-SC,20.42,93.0,-0.56,0.52,2.38,X40,NTT,FOOTWEAR
58,RELAXO,1.35,-49.87,203.80,52.31,148315.0,-72385.0,72775.0,1176.00,-46.67,50.0,X-SC,11.60,92.0,-0.49,0.52,8.22,X40N,NTT,FOOTWEAR
51,MEDANTA,-2.40,-7.08,33.54,24.08,39563.0,-8993.0,117957.0,1486.00,-14.63,48.0,X-SC,10.32,91.0,-0.23,0.85,8.04,XY24,NTT,HEALTHCARE
35,HONAUT,0.92,-16.70,72.04,43.32,97744.0,-27192.0,135680.0,58357.33,-28.43,56.0,X-SC,8.80,90.0,-0.28,0.97,10.17,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.31,-22.87,43.63,10.78,80872.0,-54960.0,185358.0,452.00,-54.23,20.0,X-LC,18.18,1.0,-0.68,1.33,0.00,X40,NTT,FMCG
75,TMPV,2.69,-23.62,61.01,22.98,105265.0,-53361.0,172537.0,600.00,-20.63,64.0,X-LC,4.41,2.0,-0.51,1.24,10.30,XY24,NTT,AUTO
73,TCS,1.54,-11.51,33.97,18.55,113699.0,-43534.0,334703.0,4311.58,-23.62,56.0,X-LC,0.63,3.0,-0.38,2.40,13.87,X40,ATH,IT
81,VBL,-1.05,-5.81,44.04,35.67,131019.0,-18343.0,297499.0,671.64,-17.02,39.0,X-LC,1.51,5.0,-0.14,2.14,7.24,X40N,ATH,FMCG
42,INFY,2.21,13.37,18.41,34.24,65306.0,41842.0,354730.0,1972.00,-11.70,56.0,X-LC,7.24,6.0,0.64,2.55,22.76,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,2.00,-31.45,93.86,32.89,49674.0,-24282.0,52924.0,1800.00,-753.40,61.0,L-MC,11.25,259.0,-0.49,0.38,45.75,XR,NTT,BANKS
67,SONACOMS,5.09,-9.19,53.16,39.09,48836.0,-9293.0,91866.0,804.02,-29.03,72.0,M-SC,2.39,227.0,-0.19,0.66,29.96,AR,ATH,AUTO
14,BSOFT,4.71,-19.21,86.65,50.79,98459.0,-27025.0,113628.0,831.70,3.59,61.0,H-SC,6.15,171.0,-0.27,0.82,32.30,XR,ATH,IT
79,VAIBHAVGBL,11.77,-22.95,108.66,60.77,153020.0,-41950.0,140825.0,521.00,62.53,65.0,H-SC,4.45,158.0,-0.27,1.01,27.67,XR,NTT,JEWELLERY
18,COALINDIA,0.39,4.55,14.04,19.23,22086.0,6846.0,157306.0,484.83,22.81,52.0,L-LC,12.49,182.0,0.31,1.13,26.79,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,3.81,4.15,39.77,45.57,108362.0,10853.0,272472.0,7934.00,-33.07,70.0,H-MC,6.08,121.0,0.10,1.96,21.00,AR,NTT,ELECTRICAL
67,SONACOMS,5.09,-9.19,53.16,39.09,48836.0,-9293.0,91866.0,804.02,-29.03,72.0,M-SC,2.39,227.0,-0.19,0.66,29.96,AR,ATH,AUTO
5,ANGELONE,7.66,5.86,17.32,24.20,55970.0,17900.0,323150.0,3033.00,50.86,54.0,X-SC,2.00,99.0,0.32,2.32,35.34,X40N,NTT,FINANCE
27,GILLETTE,1.75,4.38,26.86,32.42,71184.0,11124.0,265020.0,11206.78,-14.09,66.0,X-SC,12.36,87.0,0.16,1.90,20.14,X40,ATH,FMCG
79,VAIBHAVGBL,11.77,-22.95,108.66,60.77,153020.0,-41950.0,140825.0,521.00,62.53,65.0,H-SC,4.45,158.0,-0.27,1.01,27.67,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.80
1,25,44.79
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.26
MC    30.49
LC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.33
X40      23.79
X40N     15.05
AR        8.99
XR        8.87
XY25      8.65
OX40N     7.41
SR        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.81
H-SC    23.04
X-LC    20.76
M-SC    11.12
X-SC    10.35
H-MC     4.83
M-MC     1.47
M-LC     1.22
H-LC     1.16
L-LC     1.13
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.98,-13.45,52.13
FINANCE,13.33,-12.55,56.87
IT,12.92,-18.85,78.23
MISC,6.99,-33.35,87.90
ELECTRICAL,6.07,-11.07,51.59
PAINTS,5.32,-24.69,43.22
INSURANCE,4.81,-1.38,36.07
PHARMA,3.98,-5.98,39.33
AUTO,2.95,-27.82,72.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3306726.0,21
AR,1320286.0,10
XR,1259482.0,12
X40,1195501.0,15
X40N,1024206.0,10
OX40N,760756.0,10
XY25,415037.0,6
SR,290957.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3588147.0,24
X-MC,1714704.0,16
M-SC,1512074.0,15
X-LC,1000733.0,12
X-SC,856399.0,9
H-MC,406326.0,3
L-SC,186702.0,2
M-LC,122566.0,1
H-LC,72709.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233107.0      6
           AR         931470.0      5
M-SC       XY24       860602.0      6
H-SC       XR         809753.0      7
X-MC       X40        579902.0      7
           XY24       480175.0      3
X-LC       X40        442494.0      6
X-MC       X40N       421747.0      4
X-SC       X40N       374503.0      4
M-SC       OX40N      345545.0      5
H-SC       OX40N      322860.0      4
X-SC       XY24       308791.0      3
H-SC       SR         290957.0      2
X-MC       XY25       232880.0      2
X-LC       X40N       227956.0      2
H-MC       XY24       213008.0      1
X-LC       XY24       211043.0      2
H-MC       AR         193318.0      2
M-SC       XR         183138.0      2
X-SC       X40        173105.0      2
M-SC       AR         122789.0      2
M-LC       XR         122566.0      1
X-LC       XY25       119240.0      2
L-SC       XR          94351.0      1
           OX40N       92351.0      1
H-LC       AR          72709.0      1
L-MC       XR          49674.0      1
M-MC       XY25        40831.0      1
L-LC       XY25        22086.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
